In [ ]:
import os

# params that need to be set for each computing node
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
gpu_avail = 12
base_folder = "/raid/8wiehe/"

In [ ]:
import sys
import gc
import shutil

import torch
import numpy as np


PYTHONPATH = sys.executable

# set song
song_name = "songs/Daft_Punk_Harder_Better_Faster.mp3"
#song_name = "songs/vbnd_daughter_of_the_sun.mp3"
#song_name = "free_songs/tannenbaum_by_dan_lerch.mp3"
#song_name = "songs/ratatat_Loud_Pipes.mp3"
#song_name = "songs/gnossi_1.mp3"
#song_name = "songs/henne_song.mp3"
#song_name = "songs/Präludium_1_bach.mp3"
#song_name = "songs/Präludium_2_Bach.mp3"
#song_name = "songs/Blumen.wav"
#song_name = "songs/Misty.mp3"
#song_name = "songs/space_is_the_place_men_I_trust.mp3"
#song_name = "songs/opus_men_I_trust.mp3"
#song_name = "songs/any_colour_you_like_pink_floyd_hd_studio_quality_7032261705832661515.mp3"

# song params
offset = 0
duration = None

# set some parameters based on how much memory is available
sideX, sideY = 656, 368
clip_batch_size = 4
gpt_name = "gpt2"
if gpu_avail >= 8:
    clip_batch_size = 4
    gpt_name = "neo1.3"
elif gpu_avail >= 12:
    clip_batch_size = 4
    gpt_name = "neo2.7"
    sideX, sideY = 880, 484
# general params
fps = 30
boost_fps = 60
# substep params
sub_steps = 12  # constant substeps if variable substeps are not used
use_variable_substeps = True
min_substeps = 3
max_substeps = 15
# musicnn params
input_overlap = 1 / 30
input_length = 3
# post processing params
upscale = False
twice_upscale = False
# video params
total_effect_strength = 0.2
ema_val = 0.2 # 0.99 is too strong
ema_val_latent = 0.3

lpips_weight = 2.2  # 1.5 is good for 300 sub_steps but more could work too.
# prompts
base_img_path = "../CLIP_playground/base_images/"
# settings for prompts
k = 5
prompt_mode = "weighted_top_k" # top_k, weighted_top_k, gpt
use_mean_dirs = True
mood_weight = 0.33  # how much to weigh the current story vs the predicted "feelings" - should be between 0.1-0.4, otherwise the story disappears

prefix = ""  # prefix for ALL prompts
taggram_mode = "feelings" # full, feelings
general_theme = "" #". In the style of James Gurney."
create_gpt_artstyle = True
num_themes = 10 # num GPT themes to sample
create_clusters = True
gpt_cluster_prefix = ""  # prefix for clusters
do_create_gpt_cluster_stories = True
use_k_means = False
min_clusters = 3
max_clusters = 13
cluster_time_weight = 0.1  # how much weight to give to the time component while clustering in proportion to the maximum feature
ema_val_clustering = 0.96

gpt_story_top_k = 2  # number of k top cluster stories that will be consideren in CLIP guidance
img_theme = None # base_img_path + "hot-dog.jpg"
# base_img_path + "Autumn_1875_Frederic_Edwin_Church.jpg"
#". By Pete Mohrbacher."
#". In the style of 'The Persistence of Memory' by Dali." #" by madziowa_p." #" by Jan Brueghel the Elder."  #" by Salvador Dali." #" by Greg Rutkowski."

In [ ]:
import soundfile
import librosa
import os

resampled_path = "tmp/resampled.wav"
os.makedirs("tmp", exist_ok=True)

# load song and resample to 16k Hz
sr = 16000
raw_song, old_sr = librosa.load(song_name, offset=offset, duration=duration)
song = librosa.resample(raw_song, old_sr, sr)
soundfile.write(resampled_path, song, sr)

In [ ]:
import pandas as pd
from mustovi_utils import get_taggram
    
tag_dfs_folder = "./tmp/tag_dfs"
os.makedirs(tag_dfs_folder, exist_ok=True)
key_song_name = song_name.split("/")[-1].split(".")[0]
tag_df_name = f"{key_song_name}_{input_length}_{int(1 / input_overlap)}_{offset}_{duration}.csv"
tag_df_path = os.path.join(tag_dfs_folder, tag_df_name)
if os.path.exists(tag_df_path):
    normed_tag_df = pd.read_csv(tag_df_path, index_col=0)
else:
    normed_tag_df = get_taggram(resampled_path, input_overlap, input_length)
    normed_tag_df.to_csv(tag_df_path)
    torch.cuda.empty_cache()
    gc.collect()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# ordered by importance
show_all_labels = False
if show_all_labels:
    plt.figure(figsize=(12, 20))
    show_df = normed_tag_df.T.copy()
    show_df["mean"] = normed_tag_df.mean(axis=0)
    show_df = show_df.sort_values("mean").drop(columns=["mean"])
    sns.heatmap(show_df)
    plt.tight_layout()

In [ ]:
instruments = ["violin", "strings", "sitar", "piano", "harpsichord", 
               "harp", "guitar", "drums", "flute", "synth", "cello"]
genres = ["techno", "soul", "rock", "rnb", "punk", "pop", "opera", 
          "oldies", "new age", "metal", "jazz", "indie rock", "indie pop",
          "indie", "indian", "heavy metal", "hard rock", "funk", "folk", 
          "electronica", "electronic", "country", "classical", "classic rock",
          "classic", "choral", "blues", "alternative rock", "alternative",
          "Progressive rock", "House", "Hip-hop"]
eras = ["60s", "70s", "80s", "90s", "00s"]

speed_tags = ["fast", "slow"]
feeling_tags = ["weird", "soft", "happy", "sad", "catchy", "easy listening", "sexy", "chillout", "beautiful", "chill"]
loudness_tags = ["quiet", "loud"]
vibe_tags = ["ambient", "party", "dance", "Mellow", "experimental"]

genre_like_tags = ["solo", "blues", "Beat"]

feeling_tags = speed_tags + feeling_tags + loudness_tags + vibe_tags
    
plt.figure(figsize=(10, 7))
show_df = normed_tag_df[feeling_tags].T.copy()
show_df[show_df < show_df.mean(axis=0)] = 0
show_df["mean"] = show_df.mean(axis=1)
show_df = show_df.sort_values("mean").drop(columns=["mean"])
sns.heatmap(show_df)
plt.tight_layout()

In [ ]:
# reduce taggram to fit fps
musicnn_fps = 62.5
#averaging_window = int(musicnn_fps / fps) # == 2 - 30fps
averaging_window = int(np.round(musicnn_fps / fps)) # == 3 - 20fps
# take step average taggram
fps_taggram = normed_tag_df.rolling(averaging_window, min_periods=1, axis=0).mean() 
fps_taggram = fps_taggram.iloc[::averaging_window, :]

In [ ]:
# decide on using only subset
used_tag_df = fps_taggram.copy()
if taggram_mode == "feelings":    
    used_tag_df = used_tag_df[feeling_tags]
    
# rename some columns
rename_dict = {"sexy": "sensual",
               "party": "energetic",
               "dance": "moving",
               "easy listening": "harmonious",
               "catchy": "captivating"}
used_tag_df = used_tag_df.rename(columns=rename_dict)
# merge some columns
merge_dict = {"chill": "chillout"}
for key in merge_dict:
    val = merge_dict[key]
    used_tag_df[val] = (used_tag_df[key] + used_tag_df[val]) / 2
    del used_tag_df[key]

tag_df_means = used_tag_df.mean()
used_tag_df[used_tag_df < tag_df_means] = 0

In [ ]:
def apply_ema(arr, ema_val=0.9):
    ema = arr[0]
    out = []
    for item in arr:
        ema = ema * ema_val + item * (1 - ema_val)
        out.append(ema)
    return out

In [ ]:
import matplotlib.pyplot as plt

# create clusters
if create_clusters:
    import sklearn
    
    clustering_feats = used_tag_df.to_numpy()
    # add index to give some time continuity
    cluster_time = True
    if cluster_time:
        idx_arr = np.expand_dims(np.arange(len(clustering_feats)), 1)
        idx_arr = idx_arr / idx_arr.sum() * used_tag_df.sum().max() * cluster_time_weight
        clustering_feats = np.concatenate([clustering_feats, idx_arr], axis=1)
    # create high dim umap embeddings for clustering
    cluster_on_umap_high_d = False
    clusterable_embedding = np.array(apply_ema(clustering_feats, ema_val=ema_val_clustering))
    
    if use_k_means:
        # cluster        
        from yellowbrick.cluster import KElbowVisualizer

        # Instantiate the clustering model and visualizer
        model = clusterer = sklearn.cluster.KMeans(n_clusters=5, n_init=20, max_iter=500)
        visualizer = KElbowVisualizer(
            model, k=(min_clusters, max_clusters), metric='calinski_harabasz', #'silhouette', #'calinski_harabasz', 
            timings=False
        )

        visualizer.fit(clusterable_embedding)        # Fit the data to the visualizer
        visualizer.show()  
        num_clusters = visualizer.elbow_value_
        # make final clustering
        clusterer = sklearn.cluster.KMeans(n_clusters=num_clusters, n_init=50, max_iter=500)
        labels = clusterer.fit_predict(clusterable_embedding)
        # determine centers
        real_centers = clusterer.cluster_centers_
        dist_to_centers = np.array([np.mean((emb - real_centers) ** 2, axis=-1)
                                    for emb in clusterable_embedding[:, :]])
        dist_to_centers = torch.from_numpy(dist_to_centers)
        if cluster_time:
            centers = real_centers[:, :-1]
        else:
            centers = real_centers
        
    else:
        # cluster using hdbscan
        from hdbscan import HDBSCAN
        min_samples = 36
        hdbscan_labels = [-1]
        while len(np.unique(hdbscan_labels)) < 3:
            hdbscan_clusterer = HDBSCAN(min_samples=min_samples,  #35, 
                                        cluster_selection_epsilon=0., 
                                        min_cluster_size=min(100, len(clusterable_embedding) // 10))
            hdbscan_labels = hdbscan_clusterer.fit_predict(clusterable_embedding)
            min_samples = min_samples // 2
        # assign outliers (labels == -1) to the previous non-outlier label 
        clean_hdbscan_labels = []
        last_label = np.array(hdbscan_labels[hdbscan_labels != -1])[0]
        for label in hdbscan_labels:
            if label == -1:
                label = last_label
            else:
                last_label = label
            clean_hdbscan_labels.append(label)
        labels = np.array(clean_hdbscan_labels)
        num_labels = len(np.unique(labels))

        
        # assign distances to cluster for each point
        from hdbscan_utils import *
        data = clusterable_embedding
        tree = hdbscan_clusterer.condensed_tree_
        exemplar_dict = {c: exemplars(c, tree) for c in tree._select_clusters()}
        cluster_ids = tree._select_clusters()
        raw_tree = tree._raw_tree
        all_possible_clusters = np.arange(data.shape[0], raw_tree['parent'].max() + 1).astype(np.float64)
        max_lambda_dict = {c:max_lambda_val(c, raw_tree) for c in all_possible_clusters}

        point_dict = {c:set(points_in_cluster(c, raw_tree)) for c in all_possible_clusters}
        cluster_distances = np.array([combined_membership_vector(x, data, tree, exemplar_dict, cluster_ids,
                                                       max_lambda_dict, point_dict, False) for x in range(len(data))])
        dist_to_centers =  1 - torch.from_numpy(cluster_distances)
        # find cluster representatives by averagin the points per cluster
        real_centers = np.array([data[labels == i].mean(axis=0) for i in range(num_labels)])
        if cluster_time:
            centers = real_centers[:, :-1]
        else:
            centers = real_centers

    show_2d_umap = 0
    if show_2d_umap:
        from umap import UMAP
        # create 2D UMAP embedding to plot
        mapper = UMAP(
            n_neighbors=30,
            min_dist=0.0,
            n_components=2,
            random_state=42,
            metric="cosine",
        ).fit(clustering_feats)
        # make plot
        import umap.plot
        umap.plot.output_notebook()
        df = pd.DataFrame({"step": list(range(len(labels))),
                           "cluster": labels,
                           })
        p = umap.plot.interactive(mapper, 
                                  labels=df["cluster"], 
                                  #values = df["step"],
                                  hover_data=df, point_size=10)
        umap.plot.show(p)
    # show clusters over time
    plt.figure(figsize=(10, 7))
    plt.scatter(range(len(labels)), labels, s=1.5)
    plt.show()
    plt.close()
    # show cluster dists over time
    plt.figure(figsize=(10, 7))
    for i in range(dist_to_centers.shape[1]):
        plt.plot(dist_to_centers[:, i], label=str(i))
    l = plt.legend()
    plt.show()
    plt.close()
    # show heatmap 
    plt.figure(figsize=(10, 7))
    show_df = used_tag_df.T.copy()
    show_df[show_df < show_df.mean(axis=0)] = 0
    show_df["mean"] = show_df.mean(axis=1)
    show_df = show_df.sort_values("mean").drop(columns=["mean"])
    sns.heatmap(show_df)
    plt.tight_layout()
    plt.show()
    plt.close()

In [ ]:
import IPython

# listen to clusters

cluster_idx = 0

samples_per_step = int(len(song) / len(hdbscan_labels)) + 1
frame_assignments = []
for label in hdbscan_labels:
    frame_assignments.extend([label] * samples_per_step)
frame_assignments = np.array(frame_assignments)
sections = frame_assignments == cluster_idx
song_section = song[sections[:len(song)]]

IPython.display.Audio(song_section, rate=sr, autoplay=1)

In [ ]:
#story_idx = max(0 - n_start_prompts, 0)
story_idx = 0
top_k = dist_to_centers[story_idx].topk(k=gpt_story_top_k, largest=False)
print(dist_to_centers[story_idx])
print(top_k)
print(dist_to_centers[story_idx].max())
print((1 - (top_k.values / dist_to_centers[story_idx].sum())) ** 2)
#plt.plot((1 - (top_k.values / dist_to_centers[story_idx].max())) ** 2)

In [ ]:
def get_gpt_stories_and_weights(cluster_gpt_stories, n_start_prompts, dist_to_centers, gpt_story_top_k, idx):
    if cluster_gpt_stories is not None:
        story_idx = max(idx - n_start_prompts, 0)
        top_k = dist_to_centers[story_idx].topk(k=gpt_story_top_k, largest=False)
        story_weights = (1 - (top_k.values / dist_to_centers[story_idx].max())) ** 2
        top_idcs = top_k.indices
        gpt_stories = [cluster_gpt_stories[i] for i in top_idcs]
    else:
        gpt_stories = [""]
        story_weights = [1]
    return gpt_stories, story_weights

In [ ]:
used_tag_df.mean().sort_values()

In [ ]:
# highest_classification

def filter_theme_labels(series, n=6, threshold=0.5, factor=0.9):
    # sort by logits
    cluster_theme = series.sort_values(ascending=False)
    #print(cluster_theme)
    # filter logits below certain value
    cluster_theme = cluster_theme[cluster_theme > threshold]
    # filter logits that are lower than mean over all
    cluster_theme = cluster_theme[[col for col in cluster_theme.index if cluster_theme[col] > factor * used_tag_df.mean()[col]]]
    # pick top N
    cluster_theme = cluster_theme.iloc[:n]
    return cluster_theme


center_df = pd.DataFrame(centers, columns=used_tag_df.columns)

cluster_themes = []
for i in range(len(center_df)):
    cluster_theme = filter_theme_labels(center_df.iloc[i])
    cluster_vals = round(cluster_theme, 2).to_list()
    cluster_theme_names = cluster_theme.index.to_list()
    cluster_theme_names = ", ".join(cluster_theme_names).lower()
    print(str(i) + ":",  cluster_theme_names, cluster_vals)
    cluster_themes.append(cluster_theme_names)

In [ ]:
main_theme = used_tag_df.mean().sort_values(ascending=False).iloc[:5]
main_theme_words = ", ".join(main_theme.index.to_list())
main_theme

In [ ]:
# main distinctive features 
print(", ".join(center_df.std().sort_values(ascending=False)[:5].index.to_list()))

In [ ]:
import IPython

# listen to clusters

cluster_idx = 0

plt.scatter(range(len(labels)), labels, s=1.5)
plt.show()

samples_per_step = int(len(song) / len(labels)) + 1

frame_assignments = []
for label in labels:
    frame_assignments.extend([label] * samples_per_step)
frame_assignments = np.array(frame_assignments)

sections = frame_assignments == cluster_idx

song_section = song[sections[:len(song)]]

IPython.display.Audio(song_section, rate=sr, autoplay=0)

In [ ]:
from tqdm.auto import tqdm
import numpy as np

# create musicnn prompts
clip_prompts = []
pbar = tqdm(list(used_tag_df.iterrows()))

for i, row in pbar:
    row = row[row > tag_df_means]
    sorted_row = row.sort_values(ascending=False)

    # generate clip prompt for current musicnn targets
    if prompt_mode == "top_k":
        # get tags
        top_tag_names = list(sorted_row.iloc[:k].index)
        #print(top_tag_names)
        pbar.set_description(", ".join(top_tag_names))
        clip_prompt = ", ".join(top_tag_names)
    elif prompt_mode == "weighted_top_k":
        sorted_row = filter_theme_labels(sorted_row, n=k)#, factor=0.8, threshold=0.1)
        top_tag_names = list(sorted_row.index)
        top_tag_vals = list(sorted_row)
        clip_prompt = {name: val for name, val in zip(top_tag_names, top_tag_vals)}
    elif prompt_mode == "gpt":
        sorted_row = row.sort_values(ascending=False)
        top_tags = sorted_row.iloc[:k]
        top_tag_names = list(top_tags.index)
        if len(top_tag_names) == 0:
            top_tag_names = ["Undecided emptiness"]
        merged_top_tags = ", ".join(top_tag_names)
        if merged_top_tags in prompt_hash_table:
            clip_prompt = prompt_hash_table[merged_top_tags]
        else:
            clip_prompt = gpt_create_prompt(gpt_model, gpt_tokenizer, merged_top_tags)
            pbar.set_description("Tags: " + merged_top_tags + " Prompt: " + clip_prompt)
            #clip_encoding = imagine.create_text_encoding(clip_prompt)
            prompt_hash_table[merged_top_tags] = clip_prompt
            
    clip_prompts.append(clip_prompt)
    
# how many steps are there to fill at the start of the song (256 is the size of the fft-windows of musicnn)
start_prompt = clip_prompts[0]
n_start_prompts = int(np.round((len(song) / (256 * averaging_window) - len(used_tag_df))))
clip_prompts.extend([start_prompt] * n_start_prompts)

In [ ]:
import sys
sys.path.append("../StyleCLIP_modular")
sys.path.append("../CLIPGuidance")


import argparse
import torch
import gc

from style_clip import Imagine, create_text_path

net = "vqgan" # conv, vqgan

args = {}
args["lr_schedule"] = 0
args["seed"] = 1

args["neg_text"] = 'incoherent, confusing, cropped, watermarks'
#'text, signature, watermarks, writings, scribblings'#

args["clip_names"] = ["ViT-B/16", "ViT-B/32"]#, "RN50"]
args["averaging_weight"] = 0
args["early_stopping_steps"] = 0
args["use_tv_loss"] = 1
args["lpips_weight"] = lpips_weight

if net == "vqgan":
    args["model_type"] = "vqgan"
    args["lr"] = 0.1
    
elif net == "conv":
    args["model_type"] = "conv"
    args["act_func"] = "gelu"
    args["stride"] = 1
    args["num_layers"] = 5
    args["downsample"] = False
    args["norm_type"] = "layer"
    args["num_channels"] = 64
    args["sideX"] = 1080
    args["sideY"] = 720
    args["lr"] = 0.005
    args["stack_size"] = 4


args["batch_size"] = clip_batch_size
args["sideX"] = sideX # 688 #624 #544 #480 
args["sideY"] = sideY # 384 #352 #304 #272 
# 688x384 - 7.792GB, 34s/it
# 720x400 - 7.948GB, 41.3s/it - crashes after a bit
# 624x352 - 6.9GB, 29.8s/it
# 544x304 - 5850MB, 24s/it at 100its per step
args["circular"] = 0

imagine = Imagine(
                save_progress=False,
                open_folder=False,
                save_video=False,
                verbose=False,
                **args
               )

torch.cuda.empty_cache()
gc.collect()

In [ ]:
#clip_model = imagine.perceptor.models[0]
#gpt_model, gpt_tokenizer = load_gpt_model(gpt_name)

In [ ]:
#prefix = "The following are adjectives describing a song, followed by a description of the corresponding image:\n "
#prefix = "The following are adjectives describing an image, listed in the order of importance. They are followed by a full description of the corresponding image:\n "
#prompter = ". Full description:"
#examples = {"sad, dark, fast": " A man is running through dark woods while crying.",
#            "sad, beautiful, soft, quiet, slow": " An old woman is sitting on a chair in a beautiful garden with her hands folded in front of her. She is looking at you with a sad expression on her face.",
#            "electronic, loud, happy, abstract": " Dynamic and vibrant colors forming strong geometric shapes that resemble a rave.",
#            "weird, happy, fast": " A man is experiencing a strange dream. He is struggling to feel his feelings, his emotions as they rush too quickly through his body. He is an a state of ecstacy.",
#            "harmonious, mellow": " An electric light begins to dim at a distant point in the sky. You feel complete and at one with your environment.",
#            "slow, quiet": " A lion's roar stops in front of him. The lion is slowly moving forward and approaching you. He is silent."
#           }

#target_text = "slow, brutal"
#target_clip_feats = clip_model.encode_text(tokenize(target_text).to("cuda"))

In [ ]:
"An ancient dream starts to calm down. It is quiet and peaceful"
"An ordinary man appears before you. He is listening to you. He is a crazed crazy crazy crazy crazy crazy. He is beating on your legs"

In [ ]:
#texts = gen_sent(gpt_model, gpt_tokenizer, clip_model, target_clip_feats, 
#             start_text="", p=0.9, 
 ##            prefix=prefix, examples=examples, prompter=prompter, target_text=target_text,
 #            clip_weight=0.9, 
 #            clip_temp=0.45, gpt_temp=0.75, out_len=50, v=-1, num_beams=50, return_num=5)
#print(texts)

In [ ]:
from tqdm.auto import tqdm
from mustovi_utils import load_gpt_model, gen_sent
from clip import tokenize


def gpt_create_prompt(cluster_words_list, gpt_name, clip_model, gpt_model=None, gpt_tokenizer=None, gpt_prefix=""):
    if gpt_model is None:
        gpt_model, gpt_tokenizer = load_gpt_model(gpt_name)

    prefix = "The following are adjectives describing a song, followed by a description of the corresponding image:\n "
    prefix = "The following are adjectives describing an image, listed in the order of importance. They are followed by a full description of the corresponding image:\n "
    prompter = ". Full description:"
    #examples = {"sad, dark, fast": " A man is running through dark woods while crying.",
    #            "sad, beautiful, soft, quiet, slow": " An old woman is sitting on a chair in a beautiful garden with her hands folded in front of her. She is looking at you with a sad expression on her face.",
    #            "electronic, loud, happy, abstract": " Dynamic and vibrant colors forming strong geometric shapes that resemble a rave.",
    #           }
    examples = {"sad, dark, fast": " Running through dark woods while crying.",
            "sad, beautiful, soft, quiet, slow": " An old widow is sitting on a chair in a beautiful garden with her hands folded in front of her. She is looking at you with a sad expression on her face.",
            "electronic, loud, happy, abstract": " Dynamic and vibrant colors are forming strong geometric shapes that resemble a rave.",
            "weird, happy, fast": " A man is experiencing a strange dream. He is struggling to feel his emotions as they rush too quickly through his body. He is an a state of ecstacy.",
            "harmonious, mellow": " An electric light begins to dim at a distant point in the sky. You feel complete and at one with your environment.",
            "slow, quiet": " A lion's roar stops in front of him. The lion is slowly moving forward and approaching you. He is silent."
           }

    
    gpt_stories = []
    for target_text in tqdm(cluster_words_list):
        target_clip_feats = clip_model.encode_text(tokenize(target_text).to("cuda"))

        texts = gen_sent(gpt_model, gpt_tokenizer, clip_model, target_clip_feats, 
                 start_text=gpt_prefix, p=0.9, 
                 prefix=prefix, examples=examples, prompter=prompter, target_text=target_text,
                 clip_weight=0.4, 
                 clip_temp=0.45, gpt_temp=0.75, out_len=50, v=-1, num_beams=50, return_num=1)
        text = texts[0]
        gpt_stories.append(text)
        print(text)
        print()
    gpt_model = gpt_model.to("cpu")
    return gpt_stories

used_gpt_stories = None
if do_create_gpt_cluster_stories:
    used_gpt_stories = gpt_create_prompt(cluster_themes, gpt_name, imagine.perceptor.models[0], gpt_prefix=gpt_cluster_prefix)
torch.cuda.empty_cache()
gc.collect()

In [ ]:
cluster_themes

In [ ]:
used_gpt_stories

In [ ]:
from mustovi_utils import load_gpt_model, gen_sent
from clip import tokenize


def gpt_create_theme(theme_words, gpt_name, clip_model, gpt_model=None, gpt_tokenizer=None):
    if gpt_model is None:
        gpt_model, gpt_tokenizer = load_gpt_model(gpt_name)

    prefix = "The following are adjectives, followed by a matching artstyle:\n "
    prompter = ". Matching artstyle:"
    
    prompter = "The name of a matching painter is:"
    prefix= "The following are lists of words describing art, followed by the name of the artist:\n "
    
    prefix = "The following are lists of words describing art, followed by the name of the artist:\n "
    prompter = "Matching visual artist:"
    
    prefix = "The following are lists of adjectives, listed in order of importance. They are followed by a name of an artstyle that matches them:\n "
    prompter = ". Matching artstyle:"
        
    examples = {"introspective, beautiful, sad": " A moody, ambient painting.",
                "expressive, wild, colourful": " An expressionist piece of art.",
                "epic, fantasy, stunning, moody": " Illustrated by Greg Rutkowski.",
                "introspective, beautiful, sad": " Impressionism.",
                "realistic, beautiful, landscapes, forgotten civilizations": " By James Gurney."}
    # popular, internet{prompter} Trending on artstation.
    # rendered, detailed, high-quality{prompter} Rendered in unreal engine.
    # expressionist, beautiful, vibrant. {prompter} Van Gogh.
    # happy, dreamy, romantic, sensual. {prompter} Gustav Klimt.
    
    target_clip_feats = clip_model.encode_text(tokenize(theme_words).to("cuda"))

    texts = gen_sent(gpt_model, gpt_tokenizer, clip_model, target_clip_feats, 
                 start_text="", p=0.9, 
                 prefix=prefix, examples=examples, prompter=prompter, target_text=theme_words,
                 clip_weight=0.2, 
                 clip_temp=0.45, gpt_temp=0.75, out_len=50, v=-1, num_beams=50, return_num=5)    
    text = texts[0]
    print(texts)
    gpt_model = gpt_model.to("cpu")
    return text

gpt_theme = ""
if create_gpt_artstyle:
    gpt_theme = gpt_create_theme(main_theme_words.lower(), gpt_name, imagine.perceptor.models[0])
torch.cuda.empty_cache()
gc.collect()
gpt_theme

In [ ]:
gpt_theme

In [ ]:
clip_prompts[0:5]

In [ ]:
imagine = imagine.cuda()

In [ ]:
len(clip_prompts)

In [ ]:
# Calculate encodings based on prompts

clip_target_encodings = []
clip_feature_hash_table = dict()
gpt_suffix = "" if len(gpt_theme) == 0 else f" {gpt_theme}"

count = []

def encode(prompt):
    prompt = prefix + prompt
    if general_theme is not None:
        prompt = prompt + general_theme
    prompt += gpt_suffix
    if prompt in clip_feature_hash_table:
        encoding = clip_feature_hash_table[prompt]
    else:
        count.append(0)
        if len(count) % 50 == 0:
            print(prompt)
        encoding = imagine.create_clip_encoding(text=prompt, img=img_theme)
        #encoding = imagine.create_text_encoding(prompt)
        clip_feature_hash_table[prompt] = encoding
    return encoding


def weighted_average_encoding(encodings, weights):
    clip_encoding = [norm(torch.stack([norm(encoding[j]) * weight for encoding, weight in zip(encodings, weights)]).sum(dim=0))
                         for j in range(len(encodings[0]))]
    return clip_encoding


def norm(a):
    return a / a.norm(dim=-1, keepdim=True)


def clip_mean_direction(direction_prompt, base_prompts, imagine):
    base_encs = [imagine.create_clip_encoding(text=p) for p in base_prompts]
    base_plus_dir_encs = [imagine.create_clip_encoding(text=p + direction_prompt) for p in base_prompts]
    diff_encs = [[norm(norm(base_ext_enc[i]) - norm(base_enc[i])) for i in range(len(base_enc))] for base_enc, base_ext_enc in zip(base_encs, base_plus_dir_encs)]
    mean_diff_enc = [norm(torch.stack([diff_encs[j][i] for j in range(len(diff_encs))]).mean(dim=0)) for i in range(len(diff_encs[0]))]
    return mean_diff_enc



if use_mean_dirs:
    base_prompts = ["A photo of ", " ", "A painting of ", "This painting is: ", "This photo looks ", "I feel ", "I feel: ", "The sky is ",
                   "This is ", "The ground is ", "This person is ", "She is ", "He  is ", "A "]
    dir_dict = {col: clip_mean_direction(col, base_prompts, imagine) for col in used_tag_df}


for idx, prompt in enumerate(tqdm(clip_prompts)):
    gpt_stories, story_weights = get_gpt_stories_and_weights(used_gpt_stories, n_start_prompts, 
                                                             dist_to_centers, gpt_story_top_k, idx)
    
    story_encodings = []
    for gpt_story, story_weight in zip(gpt_stories, story_weights):
        if isinstance(prompt, dict):
            if use_mean_dirs:
                take_avg = True
                
                clip_encoding = encode(gpt_story + " ")
                
                if take_avg:
                    dir_encodings = [dir_dict[tag] for tag in prompt]
                    weights = list(prompt.values())
                    if len(dir_encodings) > 0:                        
                        dir_mean_encoding = weighted_average_encoding(dir_encodings, weights)
                        clip_encoding = weighted_average_encoding([clip_encoding, dir_mean_encoding], [1 - mood_weight, mood_weight])
                else:
                    for tag in prompt:
                        weight = prompt[tag]
                        clip_encoding = [clip_encoding[i] + dir_dict[tag][i] * weight for i in range(len(clip_encoding))]
                clip_encoding = [norm(enc) for enc in clip_encoding]
            else:
                encodings = [encode(gpt_story + " It feels " + prompt_key + ".") for prompt_key in prompt]
                weights = list(prompt.values())
                clip_encoding = weighted_average_encoding(encodings, weights)
        else:
            story_prompt = gpt_story + " " + prompt + "."
            clip_encoding = encode(story_prompt)
        story_encodings.append(clip_encoding)
    clip_encoding = weighted_average_encoding(story_encodings, story_weights)

    
    clip_encoding = [enc.to("cpu") for enc in clip_encoding]
    clip_target_encodings.append(clip_encoding)
    
print(len(count))

In [ ]:
# test directions
"""
print(dir_dict.keys())
base_text = "Home."
steps = 150
text_weight = 0.4
dirs = ["slow", "experimental"]
weights = [1.0, 1.0, 1.0, 1.0]

text_enc = imagine.create_text_encoding(base_text)


dir_encodings = [dir_dict[tag] for tag in dirs]
print(dir_encodings[0][0][0][:10])
dir_mean_encoding = weighted_average_encoding(dir_encodings, weights)
clip_encoding = weighted_average_encoding([text_enc, dir_mean_encoding], [text_weight, 1 - text_weight])
print(clip_encoding[0][0][:10])

#for p, w in zip(dirs, weights):
#    dir_enc = dir_dict[p]
#    clip_encoding = [norm(text_enc[i] * text_weight + dir_enc[i] * (1 - text_weight)) for i in range(len(dir_enc))]

    
imagine.set_clip_encoding(encoding=clip_encoding)
imagine.reset()
for _ in tqdm(range(steps)):
    img, loss = imagine.train_step(0, 0)
"""
# to_pil(img.squeeze())

In [ ]:
clip_prompts[0]

In [ ]:
clip_prompts[:3]

In [ ]:
#clip_target_encodings[10][0][0][:10]

In [ ]:
# take ema of encodings to smoothen
ema_encodings = []
ema = clip_target_encodings[0]

for encoding in clip_target_encodings:
    ema = [ema_val * ema[i].to("cpu") + (1 - ema_val) * encoding[i].to("cpu") for i in range(len(encoding))]
    ema_encodings.append(ema)

In [ ]:
import torchvision.transforms as T
from scipy.interpolate import NearestNDInterpolator
from mustovi_utils import get_spec_norm
import librosa
# create zoom, rotate, shift effects
effects = ["zoom", "rotate", "shiftX", "shiftY", "shear"]
harm_effect_dict =dict()  #{"rotate": 0.0}
perc_effect_dict = dict() #{"zoom": -0.5}
cqt_effect_dict = [{"zoom": 1.0}, 
                   {"rotate": 1.0},
                   {"shiftX": 1.0}, 
                   {"shiftY": 1.0},
                   {"shiftY": -1.0},
                   {"shiftX": -1.0},
                   {"rotate": -1.0},
                   {"zoom": -1.0},
                  ]
# divide song in percussion and harm (might divide in pitches later)
song_harm, song_perc = librosa.effects.hpss(song)
spec_norm_harm = get_spec_norm(song_harm)
spec_norm_perc = get_spec_norm(song_perc)
# get cqt spec
n_chroma = len(cqt_effect_dict)
cqt_spec = librosa.feature.chroma_cqt(y=song, sr=sr,hop_length=256, 
                                      n_chroma=n_chroma, n_octaves=7, 
                                      bins_per_octave=n_chroma * 4, norm=None)
sns.heatmap(cqt_spec)
plt.show()
# take window averages to match video fps
N = averaging_window
spec_norm_harm = np.convolve(spec_norm_harm, np.ones(N) / N , mode='valid')[::N]
spec_norm_perc = np.convolve(spec_norm_perc, np.ones(N) /N, mode='valid')[::N]
cqt_spec = np.array([np.convolve(cqt_line, np.ones(N) / N, mode='valid')[::N] 
                     for cqt_line in cqt_spec])
# min-max norm
spec_norm_harm = (spec_norm_harm - spec_norm_harm.min()) / (spec_norm_harm.max() - spec_norm_harm.min())
spec_norm_perc = (spec_norm_perc - spec_norm_perc.min()) / (spec_norm_perc.max() - spec_norm_perc.min())
cqt_spec = (cqt_spec - cqt_spec.min()) / (cqt_spec.max() - cqt_spec.min())
# create effects
class Effect:
    def __init__(self, strength, zoom=0, rotate=0, 
                 shiftX=0, shiftY=0, shear=0):
        max_zoom = 0.22
        self.zoom = 1 + max_zoom * zoom * strength
        max_rotate = 10
        self.rotate = max_rotate * rotate * strength
        max_shift = 12
        self.shift_x = max_shift * shiftX * strength
        self.shift_y = max_shift * shiftY * strength
        
        self.shear = 0
        
    def __call__(self, img):
        # transform it
        transformed_img = T.functional.affine(img, 
                                          angle=self.rotate, 
                                          translate=(self.shift_x,
                                                     self.shift_y), 
                                          scale=self.zoom, 
                                          shear=self.shear,
                                          fill=0,
                                          interpolation=torchvision.transforms.InterpolationMode.BILINEAR
                                         )
        # fill in zeros with nearest neighbor
        data = transformed_img.numpy()
        mask = np.where(~(data == 0))
        interp = NearestNDInterpolator(np.transpose(mask), data[mask])
        image_result = interp(*np.indices(data.shape))
        return torch.from_numpy(image_result)
        

def merge_dicts(effect_dict, effect_strength_dict, amplitude):
    for key in effect_strength_dict:
        content = effect_strength_dict[key] * amplitude
        if key in effect_dict:
            effect_dict[key] += content
        else:
            effect_dict[key] = content

# create effects that directly alter the image
if args["model_type"] == "vqgan" or args["model_type"] == "image":
    effects_list = []
    for i in range(len(spec_norm_harm)):
        harm = spec_norm_harm[i]
        perc = spec_norm_perc[i]
        cqt = cqt_spec[:, i]

        effect_dict = {}
        merge_dicts(effect_dict, harm_effect_dict, harm)
        merge_dicts(effect_dict, perc_effect_dict, perc)
        for cqt_effect, cqt_amplitude in zip(cqt_effect_dict, cqt):
            merge_dicts(effect_dict, cqt_effect, cqt_amplitude)

        effect = Effect(total_effect_strength, **effect_dict)
        effects_list.append([effect])
else:
    # create effects that alter the clip target shortly
    # these effects should have a different name
    effects_list = [[] * len(spec_norm_harm)]

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
len(ema_encodings)

In [ ]:
len(effects_list)

In [ ]:
def minmax(a):
    min_ = a.min()
    return (a - min_) / (a.max() - min_)

In [ ]:
min_substeps = 10
max_substeps = 50

In [ ]:
substeps_per_step = [sub_steps] * len(ema_encodings)
if use_variable_substeps:
    # vary number of substeps to take depending on amplitude
    spec_norm = get_spec_norm(song)
    spec_norm = minmax(np.array(apply_ema(spec_norm, 0.9)))
    substeps_per_step = spec_norm * (max_substeps - min_substeps) + min_substeps 
    substeps_per_step = np.round(substeps_per_step).astype(int)

In [ ]:
print(substeps_per_step.mean())
plt.plot(substeps_per_step)

In [ ]:
import shutil

from tqdm.auto import tqdm
import torchvision
import tensorflow as tf
from IPython.display import display, clear_output


to_pil = torchvision.transforms.ToPILImage()


if len(ema_encodings) > len(effects_list):
    ema_encodings = ema_encodings[:-1]
if len(ema_encodings) < len(effects_list):
    effects_list = effects_list[:-1]
assert len(ema_encodings) == len(effects_list), f"{len(ema_encodings)}, {len(effects_list)}"


img_latents = []
save_folder = base_folder + "tmp/vid_latents/"
if os.path.exists(save_folder):
    shutil.rmtree(save_folder)
os.makedirs(save_folder, exist_ok=True)


imagine.to("cuda")
imagine.reset()
imagine.set_clip_encoding(encoding=[item.to(imagine.device) for item in ema_encodings[0]])
img, loss = imagine.train_step(0, 0)
img = img.detach().cpu()

pbar = tqdm(list(range(len(ema_encodings))))
for i in pbar:
    clip_encoding, effects = ema_encodings[i], effects_list[i]
    # apply effects
    transformed_img = img.float() 
    if img is not None and len(effects) > 0:
        for effect in effects:
            transformed_img = effect(transformed_img.cpu())
        transformed_img_normed = transformed_img.mul(2).sub(1).to(imagine.device)
        latent, _, [_, _, indices] = imagine.model.model.model.encode(transformed_img_normed)
        imagine.set_latent(latent)
    # set target encoding in CLIP
    clip_encoding = [part.to(imagine.device) for part in clip_encoding]
    imagine.set_clip_encoding(encoding=clip_encoding)
    # optimize for some steps
    for _ in range(substeps_per_step[i]):
        img, loss = imagine.train_step(0, 0, lpips_img=transformed_img.to(imagine.device))
    img = img.detach().cpu()
    # get latent of img
    latent = imagine.model.model.get_latent().detach().cpu()
    if save_folder is None:
        img_latents.append(latent)
    else:
        latent_name = str(i) + ".pt"
        torch.save(latent, save_folder + latent_name)
    # save final img
    if i % (len(ema_encodings) // 20) == 0:
        pil_img = to_pil(img.squeeze())
        display(pil_img)

#img_latents = sequential_gen(ema_encodings, effects_list)
#img_latents = parallel_gen(clip_prompts)

In [ ]:
def apply_ema_disk(in_folder, out_folder, ema_val=0.9):
    if os.path.exists(out_folder):
        shutil.rmtree(out_folder)
    os.makedirs(out_folder, exist_ok=True)
    
    num_items = len([item for item in os.listdir(in_folder) if item.endswith(".pt")])
    ema = torch.load(in_folder + "0.pt")#arr[0]
    out = []
    for item_idx in range(num_items):
        item_name = str(item_idx) + ".pt"
        # load
        item = torch.load(in_folder + item_name)
        # calc
        ema = ema * ema_val + item * (1 - ema_val)
        # store
        torch.save(ema, out_folder + item_name)

In [ ]:
# take ema of encodings to smoothen
in_folder = base_folder + "tmp/vid_latents/"
out_folder = base_folder + "tmp/vid_ema_latents/"
ema_latents = apply_ema_disk(in_folder, out_folder, ema_val=ema_val_latent)
#ema_latents = apply_ema(img_latents, ema_val=ema_val_latent)

In [ ]:
# interpolate between latents to increase fps and make video smoother
from mustovi_utils import slerp

def boost_frames(ema_latents, boost_fps, song):
    goal_frame_count = boost_fps * len(song) / 16000
    current_frame_count = len(ema_latents)
    frames_to_add = np.ceil(goal_frame_count / current_frame_count)
    if frames_to_add > 1:
        video_latents = []
        for i, latent in enumerate(ema_latents):
            if i + 1 == len(ema_latents):
                next_latent = ema_latents[i + 1]
            else:
                next_latent = ema_latents[i + 1]
            latents_to_add = [slerp(latent, next_latent, frac) 
                              for frac in np.arange(frames_to_add) / frames_to_add]
            video_latents.extend(latents_to_add)
        return video_latents
    else:
        return ema_latents
    
def boost_frames_disk(in_folder, out_folder, boost_fps, song):
    if os.path.exists(out_folder):
        shutil.rmtree(out_folder)
    os.makedirs(out_folder, exist_ok=True)
    
    num_items = len([item for item in os.listdir(in_folder) if item.endswith(".pt")])
    
    goal_frame_count = boost_fps * len(song) / 16000
    current_frame_count = num_items
    frames_to_add = np.ceil(goal_frame_count / current_frame_count)
    if frames_to_add > 1:
        video_latents = []
        count = 0
        for i in tqdm(range(num_items)):
            item_name = str(i) + ".pt"
            latent = torch.load(in_folder + item_name)
            if i + 1 == num_items:
                next_latent = latent#ema_latents[i]
            else:
                next_latent_name = str(i + 1) + ".pt"
                next_latent = torch.load(in_folder + item_name)
            latents_to_add = [slerp(latent, next_latent, frac) 
                              for frac in np.arange(frames_to_add) / frames_to_add]
            for new_latent in latents_to_add:
                item_name = str(count) + ".pt"
                torch.save(new_latent, out_folder + item_name)
                count += 1
    else:
        # rename in folder to out folder
        os.rename(input_folder, output_folder)
        
#video_latents = boost_fps(ema_latents, boost_fps, song)
in_folder = base_folder + "tmp/vid_ema_latents/"
out_folder = base_folder + "tmp/vid_boosted_latents/"
boost_frames_disk(in_folder, out_folder, boost_fps, song)

In [ ]:
import os
import shutil

from PIL import Image
import soundfile
import moviepy.editor as mpy
from datetime import datetime


def load_img_paths(root):
    paths = [os.path.join(root, f) for f in os.listdir(root)
        if f.endswith(".png") or f.endswith(".jpg")]
    paths = sorted(paths, key= lambda x: int(x.split("/")[-1].split("_")[0].split(".")[0]))
    return paths


def create_video(imgs, song, song_name, vid_name, sr, duration, bitrate="5000k"):
    os.makedirs("video_gens", exist_ok=True)
    video_path = f"video_gens/{vid_name}"
    
    if isinstance(imgs, list):
        img_len = len(imgs)
    else:
        img_len = len([img for img in os.listdir(imgs) 
                       if img.endswith(".jpg") or img.endswith(".png")])
    
    audio = mpy.AudioFileClip(song_name, fps=sr, nbytes=4)  
    if duration is not None:
        audio.set_duration(duration)
    #from moviepy.audio.AudioClip import AudioArrayClip
    #audio = AudioArrayClip(np.expand_dims(song, axis=0), fps=sr) # from a numerical array
    #audio = AudioArrayClip(np.expand_dims(song, axis=1), fps=sr)
    
    # Write temporary audio file
    #soundfile.write('tmp.wav', song, sr)
    #audio = mpy.AudioFileClip("tmp.wav", fps=sr)
    
    # Generate final video
    vid_fps = len(imgs) / audio.duration
    video = mpy.ImageSequenceClip(imgs, fps=vid_fps)
    video = video.set_audio(audio)
    
    video.write_videofile(video_path, 
                      codec="libx264",
                      fps=vid_fps,
                      #audio_codec="libmp3lame",
                      audio_codec="aac",
                      threads=10,
                      bitrate=bitrate,
                      audio_bitrate="320k",
                      preset="slow",
                     )


In [ ]:
imagine = imagine.to("cpu")
torch.cuda.empty_cache()
gc.collect()

In [ ]:
# create images and save to disk:
import torchvision
from tqdm.auto import tqdm
import numpy as np
import shutil

to_pil = torchvision.transforms.ToPILImage()
extension = ".jpg"
tmp_folder = base_folder + "tmp/vid_imgs" 
final_latents_folder = base_folder + "tmp/vid_boosted_latents/"

if os.path.exists(tmp_folder):
    shutil.rmtree(tmp_folder)
os.makedirs(tmp_folder, exist_ok=True)

gen_model = imagine.model.model
imagine.cpu()
torch.cuda.empty_cache()
gc.collect()

device = torch.device("cuda")
gen_model.to(device)


num_items = len([item for item in os.listdir(final_latents_folder) if item.endswith(".pt")])
for i in tqdm(range(num_items)):
    latent = torch.load(final_latents_folder + str(i) + ".pt")
#for i, latent in enumerate(tqdm(video_latents)):
    img = gen_model(latents=latent.to(device)).to("cpu")
    pil_img = to_pil(img.squeeze())
    pil_img.save(os.path.join(tmp_folder, f"{i}{extension}"), subsampling=0, quality=95)

In [ ]:
date_time = datetime.now().strftime("%m_%d_%H:%M")  #("%m/%d/%Y, %H:%M:%S")

video_name = f"{song_name.split('/')[-1].split('.')[0]}_{prompt_mode}_ema{ema_val}_steps{sub_steps}"
video_name += "_"+ gpt_theme.replace(" ", "_") if create_gpt_artstyle else ""
video_name += f"_{args['model_type']}_{date_time}.mp4"
#video_name = "loud_pipes_kiefer.mp4"

paths = load_img_paths(base_folder + "tmp/vid_imgs")
create_video(paths, raw_song, song_name, video_name, old_sr, duration)

In [ ]:
# Clone Real-ESRGAN and enter the Real-ESRGAN
if not os.path.exists("Real-ESRGAN"):
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd Real-ESRGAN
    # Set up the environment
    !$PYTHONPATH -m pip install basicsr facexlib gfpgan
    !$PYTHONPATH -m pip install -r requirements.txt
    !$PYTHONPATH setup.py develop
    # Download the pre-trained model
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
    %cd ..

In [ ]:
imagine = imagine.cpu()
torch.cuda.empty_cache()
gc.collect()

In [ ]:
import sys
sys.path.append("Real-ESRGAN")
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
from PIL import Image
from tqdm import tqdm
import torch
import numpy as np

import matplotlib.pyplot as plt

@torch.inference_mode()
def upscale_imgs(imgs, out_folder=None, scale=4, tile=0):
    
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, 
                    num_block=23, num_grow_ch=32, scale=scale)
    upsampler = RealESRGANer(
        scale=scale,
        model_path="Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth",
        model=model,
        tile=tile,
        tile_pad=10,
        pre_pad=0,
        half=1)
    
    outs = []
    for i, img in enumerate(tqdm(imgs)):
        if isinstance(img, str):
            img = np.array(Image.open(img))[:,:,::-1]
        
        output, _ = upsampler.enhance(img, outscale=scale)
        pil_img = Image.fromarray(output[:,:,::-1])
        
        if out_folder:
            pil_img.save(os.path.join(out_folder, f"{i}.jpg"), subsample=0, quality=95)
        else:
            outs.append(pil_img)
    return outs


import torchvision

to_tensor = torchvision.transforms.ToTensor()

@torch.inference_mode()
def upscale_imgs_custom(imgs, out_folder=None, scale=4, tile=0):
    model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64, 
                    num_block=23, num_grow_ch=32, scale=scale)
    loadnet = torch.load("Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth")
    if 'params_ema' in loadnet:
        keyname = 'params_ema'
    else:
        keyname = 'params'
    model.load_state_dict(loadnet[keyname], strict=True)
    model.eval().cuda().half()

    outs = []
    for i, img in enumerate(tqdm(imgs)):
        if isinstance(img, str):
            #img = torch.from_numpy(np.ascontiguousarray(Image.open(img))[:,:,::-1].copy()).unsqueeze(0)
            img = torch.from_numpy(np.transpose(np.array(Image.open(img))[:,:,::-1].copy(), (2, 0, 1))).float().unsqueeze(0).to("cuda")
        
        output = model(img.half()).cpu().squeeze().permute(1, 2, 0).numpy()
        #output, _ = upsampler.enhance(img, outscale=scale)
        output = (output * 255.0).round().astype(np.uint8)
        pil_img = Image.fromarray(output[:,:,::-1])
        
        if out_folder:
            pil_img.save(os.path.join(out_folder, f"{i}.jpg"), subsample=0, quality=95)
        else:
            outs.append(pil_img)
    return outs

In [ ]:
import imageio

def load_images_from_mp4(path):
    vid = imageio.get_reader(path,  'ffmpeg')
    imgs = [np.array(image) for image in vid.iter_data()]
    return imgs

In [ ]:
import os
import shutil

upscale = True

if upscale:
    input_folder = base_folder + "tmp/vid_imgs"
    output_folder = base_folder + "tmp/upscaled_vid_imgs"
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)
    
    print("Upscaling...")
    
    #video_path = "video_gens/any_colour_you_like_pink_floyd_hd_studio_quality_7032261705832661515_top_k_ema0.2_steps100Weird_and_beautiful._vqgan_11_20_14:02.mp4"
    #video_name = video_path.split("/")[0]
    #input_paths = load_images_from_mp4(video_name)
    
    input_paths = load_img_paths(input_folder)
    upscale_imgs(input_paths, out_folder=output_folder, scale=4)
    #!CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py --model_path Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth --input $input_folder --output $output_folder  --netscale 4 --outscale 4 --half --face_enhance > /dev/null
    print("Done!")
    # edit name
    upscaled_video_name = video_name.split("/")
    upscaled_video_name[-1] = "HD_" + upscaled_video_name[-1]
    upscaled_video_name = "/".join(upscaled_video_name)
    # create video
    paths = load_img_paths(output_folder)    
    create_video(paths, raw_song, song_name, upscaled_video_name, old_sr, duration, bitrate="12000k")

In [ ]:
if twice_upscale:
    torch.cuda.empty_cache()
    gc.collect()
    
    input_folder = base_folder + "tmp/upscaled_vid_imgs"
    output_folder = base_folder + "tmp/twice_upscaled_vid_imgs"
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)    
    print("Upscaling...")
    input_paths = load_img_paths(input_folder)
    upscale_imgs(input_paths, out_folder=output_folder, scale=4)
    #!CUDA_VISIBLE_DEVICES=0 python Real-ESRGAN/inference_realesrgan.py --model_path Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth --input  $input_folder --output $output_folder  --netscale 2 --outscale 2 --half --face_enhance > /dev/null
    print("Done!")
    # edit name
    twice_upscaled_video_name = upscaled_video_name.split("/")
    twice_upscaled_video_name[-1] = "Full" + twice_upscaled_video_name[-1]
    twice_upscaled_video_name = "/".join(twice_upscaled_video_name)
    # create video
    paths = load_img_paths(output_folder)
    create_video(paths, song_name, twice_upscaled_video_name, old_sr)